In [ ]:

import tushare as ts
import pandas as pd
import time
from tqdm import t

# 初始化Tushare
pro = ts.pro_api('your token')

# 获取全市场股票并筛选电气设备行业
df_stocks = pro.stock_basic(exchange='', list_status='L', 
                          fields='ts_code,name,industry,market,list_date')
electrical_codes = df_stocks[df_stocks['industry'] == '电气设备']['ts_code'].tolist()


In [84]:
# 定义2024年各月
months_2024 = [(f'2024{month:02d}01', f'2024{month:02d}31') for month in range(1, 13)]

# 获取日线行情数据
def get_daily_data():
    print("\n开始获取2024年日线行情数据(daily)...")
    daily_fields = 'ts_code,trade_date,open,high,low,close,vol,amount'
    df_daily_list = []
    
    for start_date, end_date in tqdm(months_2024, desc="按月获取日线数据"):
        monthly_data = []
        try:
            # 分批获取(每次50只股票)
            batch_size = 50
            for i in range(0, len(electrical_codes), batch_size):
                batch = electrical_codes[i:i+batch_size]
                
                temp_df = pro.daily(
                    ts_code=','.join(batch),
                    start_date=start_date,
                    end_date=end_date,
                    fields=daily_fields
                )
                
                if not temp_df.empty:
                    monthly_data.append(temp_df)
                
                time.sleep(0.5)  # 控制请求频率
            
            if monthly_data:
                month_df = pd.concat(monthly_data)
                df_daily_list.append(month_df)
                print(f"{start_date[:6]}月获取{len(month_df)}条记录")
                
        except Exception as e:
            print(f"获取{start_date[:6]}月数据出错: {str(e)}")
            time.sleep(3)
    
    if df_daily_list:
        df_daily = pd.concat(df_daily_list).drop_duplicates()
        df_daily.sort_values(['ts_code', 'trade_date'], inplace=True)
        print(f"\n日线数据获取完成，共{len(df_daily)}条记录")
        return df_daily
    else:
        print("未能获取日线数据")
        return pd.DataFrame()

# 获取并保存日线数据
df_daily = get_daily_data()
df_daily.to_csv('electrical_daily_2024.csv', index=False)
df_daily.head()


开始获取2024年日线行情数据(daily)...


按月获取日线数据:   8%|▊         | 1/12 [00:05<00:55,  5.02s/it]

202401月获取6882条记录


按月获取日线数据:  17%|█▋        | 2/12 [00:09<00:49,  4.96s/it]

202402月获取4724条记录


...

按月获取日线数据:  92%|█████████▏| 11/12 [00:54<00:04,  4.99s/it]

202411月获取6682条记录


按月获取日线数据: 100%|██████████| 12/12 [01:00<00:00,  5.00s/it]

202412月获取6996条记录

日线数据获取完成，共76749条记录


,ts_code,trade_date,open,high,low,close,vol,amount
1029,000009.SZ,20240102,11.76,11.78,11.66,11.69,104077.22,121760.593
980,000009.SZ,20240103,11.66,11.73,11.61,11.70,91791.57,107089.530
931,000009.SZ,20240104,11.62,11.72,11.57,11.64,90231.12,104971.672
882,000009.SZ,20240105,11.65,11.97,11.64,11.78,191617.50,226544.734
833,000009.SZ,20240108,11.74,11.79,11.53,11.55,174964.76,202886.483


In [85]:
# 获取基本面数据
def get_daily_basic_data():
    print("\n开始获取2024年基本面数据(daily_basic)...")
    basic_fields = 'ts_code,trade_date,close,turnover_rate,pe_ttm,pb,total_mv,circ_mv'
    df_basic_list = []
    
    for start_date, end_date in tqdm(months_2024, desc="按月获取基本面数据"):
        monthly_data = []
        try:
            # 获取该月交易日历
            trade_cal = pro.trade_cal(
                exchange='', 
                start_date=start_date, 
                end_date=end_date, 
                is_open='1'
            )
            trade_dates = trade_cal['cal_date'].tolist()
            
            # 逐日获取数据
            for trade_date in tqdm(trade_dates, desc=f"{start_date[:6]}交易日", leave=False):
                # 分批获取(每次50只股票)
                batch_size = 50
                for i in range(0, len(electrical_codes), batch_size):
                    batch = electrical_codes[i:i+batch_size]
                    
                    temp_df = pro.daily_basic(
                        ts_code=','.join(batch),
                        trade_date=trade_date,
                        fields=basic_fields
                    )
                    
                    if not temp_df.empty:
                        monthly_data.append(temp_df)
                    
                    time.sleep(0.5)  # 控制请求频率
            
            if monthly_data:
                month_df = pd.concat(monthly_data)
                df_basic_list.append(month_df)
                print(f"{start_date[:6]}月获取{len(month_df)}条记录")
                
        except Exception as e:
            print(f"获取{start_date[:6]}月数据出错: {str(e)}")
            time.sleep(3)
    
    if df_basic_list:
        df_basic = pd.concat(df_basic_list).drop_duplicates()
        df_basic.sort_values(['ts_code', 'trade_date'], inplace=True)
        print(f"\n基本面数据获取完成，共{len(df_basic)}条记录")
        return df_basic
    else:
        print("未能获取基本面数据")
        return pd.DataFrame()

# 获取并保存基本面数据
df_basic = get_daily_basic_data()
df_basic.to_csv('electrical_daily_basic_2024.csv', index=False)
df_basic.head()



开始获取2024年基本面数据(daily_basic)...


按月获取基本面数据:   8%|▊         | 1/12 [01:40<18:23, 100.28s/it]

202401月获取6882条记录


按月获取基本面数据:  17%|█▋        | 2/12 [02:48<13:36, 81.67s/it] 

202402月获取4724条记录


...

按月获取基本面数据:  92%|█████████▏| 11/12 [17:02<01:34, 94.46s/it]

202411月获取6682条记录


按月获取基本面数据: 100%|██████████| 12/12 [18:42<00:00, 93.51s/it]


202412月获取6996条记录

基本面数据获取完成，共76749条记录


,ts_code,trade_date,close,turnover_rate,pe_ttm,pb,total_mv,circ_mv
0,000009.SZ,20240102,11.69,0.4079,23.9376,3.0625,3.015101e+06,2.982622e+06
0,000009.SZ,20240103,11.70,0.3598,23.9581,3.0652,3.017680e+06,2.985173e+06
0,000009.SZ,20240104,11.64,0.3536,23.8352,3.0494,3.002205e+06,2.969865e+06
0,000009.SZ,20240105,11.78,0.7510,24.1219,3.0861,3.038314e+06,3.005585e+06
0,000009.SZ,20240108,11.55,0.6858,23.6510,3.0259,2.978992e+06,2.946902e+06
